In [11]:
import pandas as pd
import numpy as np
PoliceIncident_Original=pd.read_csv("data/Police_Incident_Reports.csv")
Location = PoliceIncident_Original["Location"].copy()
Date_Occured = PoliceIncident_Original["Date Occured"].copy()
Date_Reported = PoliceIncident_Original["Date Reported"].copy()
PoliceIncident=PoliceIncident_Original.drop(["Case Status","Police Case Number",'Offense Description',"Date Found","Offense Code","Subdivision","Date Reported","Location"],axis=1)

LongLatt = r'(\d+\.\d*\,\s\-\d+\.\d*)'
Location_Inc=Location.str.extract(LongLatt, expand=False).str.rsplit(",",expand=True)
Lattitude_Inc=pd.DataFrame(Location_Inc[0]).astype(float)
Longitude_Inc=pd.DataFrame(Location_Inc[1]).astype(float)
PoliceIncident=pd.merge(PoliceIncident,Lattitude_Inc,how='left',left_index=True,right_index=True)
PoliceIncident=pd.merge(PoliceIncident,Longitude_Inc,how='left',left_index=True,right_index=True)

IncidentType=PoliceIncident_Original['Offense Description'].str.split(',',expand=True,n=1)
threshold = 500 # Anything that occurs less than this will be replaced.
value_counts = IncidentType[0].value_counts() # Entire DataFrame 
to_remove = value_counts[value_counts <= threshold].index
IncidentType[0].replace(to_remove, 'Others', inplace=True)
IncidentType[0].replace(["DRUG/ NARCOTIC VIOLATIONS","DRUG/ EQUIPMENT VIOLATIONS","OVERDOSE"], 'DRUG VIOLATIONS', inplace=True)
#["DRUG/ NARCOTIC VIOLATIONS","DRUG/ EQUIPMENT VIOLATIONS"]->'DRUG VIOLATIONS'
IncidentType[0].replace(["BURGLARY/ B & E","MOTOR VEHICLE THEFT"], 'LARCENY', inplace=True)
# ["BURGLARY/ B & E","MOTOR VEHICLE THEFT"]->'LARCENY'
IncidentType[0].replace(["WEAPON LAW VIOLATIONS","WEAPON LAW"], 'WEAPON LAW VIOLATIONS', inplace=True)
#"WEAPON LAW VIOLATIONS","WEAPON LAW"]->'WEAPON LAW VIOLATIONS'
IncidentType[0].replace(["HIT & RUN"], 'ASSAULT', inplace=True)
#["HIT & RUN"]-> 'ASSAULT
IncidentType[0].replace(["THREATEN BODILY HARM",'ANNOYING PHONE CALLS/ HARASSING COMMUNICATION'],"HARASSING COMMUNICATION", inplace=True)
#["THREATEN BODILY HARM",'ANNOYING PHONE CALLS/ HARASSING COMMUNICATION']->"HARASSING COMMUNICATION",

IncidentType=pd.DataFrame(IncidentType[0])

#from sklearn.preprocessing import LabelEncoder
#encoder = LabelEncoder()
#PoliceIncident_cat_encoded = encoder.fit_transform(IncidentType)
#Type=pd.DataFrame(PoliceIncident_cat_encoded)
##Type[0].replace({0:1,4:1,10:2,7:2,6:2,11:2,16:2,12:2,5:3,8:3,13:3,2:3,3:3,17:3,18:3,15:4,20:4,19:4,1:5,9:5,14:5}, inplace=True)
#PoliceIncident=pd.merge(PoliceIncident,Type,how='left',left_index=True,right_index=True)
Type=pd.DataFrame(IncidentType)
PoliceIncident=pd.merge(PoliceIncident,Type,how='left',left_index=True,right_index=True)

PoliceIncident.rename(columns={'0_x':'Lattitude', 1:'Longitude','0_y':'Type', "Date Occured":'Date_Occured'}, inplace=True)
PoliceIncident['Zone ID'].apply(pd.to_numeric)

Date_Reported = pd.to_datetime(Date_Reported)
Reported=((Date_Reported.dt.year*12+Date_Reported.dt.month)*30+Date_Reported.dt.day)*24 + Date_Reported.dt.hour
Reported_day=((Date_Reported.dt.year*12+Date_Reported.dt.month)*30+Date_Reported.dt.day)
PoliceIncident.Date_Occured = pd.to_datetime(PoliceIncident.Date_Occured)
Occur=((PoliceIncident.Date_Occured.dt.year*12+PoliceIncident.Date_Occured.dt.month)*30+PoliceIncident.Date_Occured.dt.day)*24 + PoliceIncident.Date_Occured.dt.hour
#Occur_day=((Date_Occured.dt.year*12+Date_Occured.dt.month)*30+Date_Occured.dt.day)
PoliceIncident['Time_Intrvl'] =Reported-Occur
#PoliceIncident['Time_Intrvl_day'] =Reported_day-Occur_day
#PoliceIncident['Time_Intrvl'].where(PoliceIncident['Time_Intrvl'] > -1, np.nan, inplace=True)


#Time_Occured=pd.DataFrame(Date_Occured.dt.hour).astype(float)
#PoliceIncident=pd.merge(PoliceIncident,Time_Occured,how='left',left_index=True,right_index=True)
#Time_Occured_day=pd.DataFrame(Date_Occured.dt.day).astype(float)
#PoliceIncident=pd.merge(PoliceIncident,Time_Occured_day,how='left',left_index=True,right_index=True)

PoliceIncident=PoliceIncident.drop([27743, 63611,13603,61541,81206,41997,35420,20130,48317,67191,33993,36432,23839,26143,39922,82728,79301,79299])
PoliceIncident=PoliceIncident.dropna(axis=0,how='any')
#PoliceIncident.drop(columns=["'Date Occured_x','Date Occured_y','Time_Intrvl_day','Lattitude','Longitude'])

In [12]:
PoliceIncident.head()

,Date_Occured,Zone ID,Lattitude,Longitude,Type,Time_Intrvl
0,2017-01-08 17:00:00,226.0,36.848522,-75.990597,DESTRUCTION OF PROPERTY,215
1,2016-06-29 07:27:00,127.0,36.828849,-76.069027,LARCENY,222
2,2016-07-15 22:40:00,227.0,36.848395,-75.975093,ASSAULT,63
3,2017-07-23 21:00:00,320.0,36.838597,-76.070625,DESTRUCTION OF PROPERTY,20
4,2017-03-05 12:55:00,124.0,36.763408,-76.021602,LARCENY,2


In [13]:
Type=pd.DataFrame(IncidentType)
PoliceIncident=pd.merge(PoliceIncident,Type,how='left',left_index=True,right_index=True)

In [14]:
PoliceIncident.to_csv('data/police_result.csv',index=False,sep=',')

In [5]:
import matplotlib.pyplot as plt
import numpy as np
# PoliceIncident.plot(kind="scatter", x="Longitude", y="Lattitude",alpha=0.1)
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 0.7, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)
if __name__ == '__main__':
    N =10
    PoliceIncident.plot(kind="scatter", x="Longitude", y="Lattitude",alpha=0.4,s=PoliceIncident['Time_Intrvl']/500, label="Time_Intrvl" ,c="Type",cmap=discrete_cmap(N, 'cubehelix'), colorbar=True,figsize=(8,4))

# PoliceIncident.plot(kind="scatter", x="Longitude", y="Lattitude",alpha=0.2,s=PoliceIncident['Time_Intrvl']/40, label="Time_Intrvl" ,c="Zone ID",cmap=plt.get_cmap('RdBu', 20), colorbar=True,figsize=(20,14))
# plt.legend()

C:\Users\user\Anaconda3\lib\site-packages\matplotlib\collections.py:902: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor


In [11]:
#from sklearn.model_selection import train_test_split
#P_train_set, P_test_set = train_test_split(PoliceIncident, test_size=0.2, random_state=42)


#corr_matrix = P_train_set.corr()
#corr_matrix["Date Occured"].sort_values(ascending=False)
#IncidentType[0].value_counts()

KeyError: 'P_train_set.Date_Occured'

In [7]:
PoliceIncident['Time_Intrvl'].value_counts()
PoliceIncident
# pd.value_counts(PoliceIncident_cat_encoded , sort=True)
# PoliceIncident_cat_encoded

,Zone ID,Case Status,Lattitude,Longitude,Type,Time_Intrvl,Time_Intrvl_day,Date Occured_x,Date Occured_y
0,226.0,INACTIVE - PENDING,36.848522,-75.990597,5,215.0,9,17.0,8.0
1,127.0,INACTIVE - PENDING,36.828849,-76.069027,10,222.0,9,7.0,29.0
2,227.0,INACTIVE - PENDING,36.848395,-75.975093,1,63.0,3,22.0,15.0
3,320.0,INACTIVE - PENDING,36.838597,-76.070625,5,20.0,1,21.0,23.0
4,124.0,INACTIVE - PENDING,36.763408,-76.021602,10,2.0,0,12.0,5.0
5,228.0,CLEARED BY ARREST,36.823212,-75.977462,8,541.0,22,0.0,25.0
6,331.0,INACTIVE - PENDING,36.906650,-76.141786,10,14.0,1,18.0,18.0
7,426.0,INACTIVE - PENDING,36.799177,-76.124827,8,61634.0,2568,8.0,25.0
9,426.0,"UNFOUNDED, NO FURTHER ACTION NEEDED",36.809335,-76.121166,13,1.0,0,4.0,20.0
10,323.0,CLEARED BY ARREST,36.843743,-76.142357,10,1.0,0,16.0,4.0


In [ ]:
PoliceIncident.reindex(PoliceIncident['Longitude'].abs().sort_values(ascending=False).index)
# Lattitude=pd.DataFrame(Location[0]).astype(float) 
# Lattitude.dtypes 
# PoliceCall.info()
PoliceIncident['Zone ID'].value_counts()


PoliceCall_Original=pd.read_csv("datasets/Police_Calls_for_Service.csv")
Location = PoliceCall_Original["Location"].copy()
CallTime = PoliceCall_Original["Call Date/Time"].copy()
CloseTime = PoliceCall_Original["Close Date/Time"].copy()
PoliceCall=PoliceCall_Original.drop(["Incident Number","Report Number","On Scene Date/Time","Entry Date/Time","Dispatch Date/Time","En Route Date/Time","Subdivision"],axis=1)

LongLatt = r'(\d+\.\d*\,\s\-\d+\.\d*)'
Location=Location.str.extract(LongLatt, expand=False).str.rsplit(",",expand=True)
Lattitude=pd.DataFrame(Location[0]).astype(float)
Longitude=pd.DataFrame(Location[1]).astype(float)
PoliceCall=pd.merge(PoliceCall,Lattitude,how='left',left_index=True,right_index=True)
PoliceCall=pd.merge(PoliceCall,Longitude,how='left',left_index=True,right_index=True)
PoliceCall.rename(columns={0:'Lattitude', 1:'Longitude'}, inplace=True)

# Time_Regular=r'(\d+\:\d+\:\d+)'
# HourMinute_OnScene=OnSceneTime.str.extract(Time_Regular, expand=False)
# HourMinute_Call=CallTime.str.extract(Time_Regular, expand=False)
# AMPM_Regular=r'(AM|PM)'
# AMPM_OnScene=OnSceneTime.str.extract(AMPM_Regular, expand=False)
# AMPM_Call=CallTime.str.extract(AMPM_Regular, expand=False)

Call_time = pd.to_datetime(CallTime)
Call=((Call_time.dt.day*24+Call_time.dt.hour)*60+Call_time.dt.minute)*60 + Call_time.dt.second
Close_time = pd.to_datetime(CloseTime)
Close=((Close_time.dt.day*24+Close_time.dt.hour)*60+Close_time.dt.minute)*60 + Close_time.dt.second
PoliceCall['Time_Intrvl'] =Close-Call

Property_Original=pd.read_csv("datasets/Property_Sales.csv")
Location_P = Property_Original["Location 1"].copy()
Property= pd.DataFrame(Property_Original,columns =["Total Value","Location 1"])

Location=Location_P.str.extract(LongLatt, expand=False).str.rsplit(",",expand=True)
Lattitude_P=pd.DataFrame(Location[0])
Longitude_P=pd.DataFrame(Location[1])
Property=pd.merge(Property,Lattitude_P,how='left',left_index=True,right_index=True)
Property=pd.merge(Property,Longitude_P,how='left',left_index=True,right_index=True)
Property.rename(columns={0:'Lattitude', 1:'Longitude'}, inplace=True)